In [1]:
#import libraries
import numpy as np
import pandas as pd
import tflearn # tensorflow deep learning
import tensorflow as tf
import random
import json
import pickle
import nltk # natural langauge tool kit for nlp

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to C:\Users\shivansh
[nltk_data]     uppal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
with open(r'C:\programming languages\CYBERCUP\CyberCup\manya\data.json', 'r') as f:
    dataset = json.load(f)

In [4]:
#converting dictinary to pandas dataframe
df = pd.DataFrame.from_dict(dataset['data'])
df.head()

,tag,patterns,responses
0,Fire,"[I am stuck in fire, Fire Fire Fire, Flames al...",[Please consider the following measures- \n 1....
1,Earthquake,"[I am in an earthquake, objects are falling, g...",[Please consider the following measures- \n 1....
2,Flood,"[Lots of water around me, neighbourhood is flo...",[Please consider the following measures- \n 1....


In [5]:
#prepare training data
words = []
labels = []
docs_x = []
docs_y = []

for data in dataset["data"]: # dataset is dict. 'data' is key. dataset['data'] is array of dict. data is dict
    for pattern in data["patterns"]: #pattern is array element in dict element of array of dict
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        docs_x.append(w)
        docs_y.append(data["tag"])

    if data["tag"] not in labels:
        labels.append(data["tag"])

ignore_words = ['?', '.', '!']
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
print(words)

["'s", ',', '[', ']', 'a', 'al', 'alarm', 'alert', 'am', 'an', 'and', 'ar', 'around', 'at', 'auth', 'bed', 'been', 'brigad', 'build', 'burn', 'cal', 'can', 'car', 'caught', 'chief', 'chok', 'clos', 'contact', 'crack', 'dam', 'depart', 'develop', 'district', 'door', 'dril', 'drown', 'earthquak', 'emerg', 'engin', 'escap', 'everyth', 'everywh', 'exit', 'explod', 'extinct', 'fal', 'fast', 'fil', 'fir', 'firefight', 'firem', 'flam', 'flood', 'for', 'from', 'ground', 'has', 'hav', 'hazard', 'heavy', 'help', 'her', 'high', 'hom', 'hous', 'hydr', 'i', 'immedy', 'in', 'into', 'is', 'it', 'lan', 'let', 'light', 'lot', 'me', 'mov', 'my', 'nee', 'neighbo', 'next', 'not', 'object', 'of', 'op', 'outsid', 'patrol', 'pay', 'peopl', 'phon', 'pleas', 'protect', 'rainfal', 'reach', 'rescu', 'road', 'room', 'rupt', 'serv', 'shak', 'smok', 'spread', 'standstil', 'stat', 'stop', 'stuck', 'submerg', 'surround', 'swing', 'the', 'to', 'trig', 'und', 'unstop', 'urg', 'us', 'very', 'volunt', 'wat', 'we', 'with'

In [6]:
len(words)

123

In [7]:
labels = sorted(labels)
print(labels)

['Earthquake', 'Fire', 'Flood']


In [8]:
print(docs_x)

[['I', 'am', 'stuck', 'in', 'fire'], ['Fire', 'Fire', 'Fire'], ['Flames', 'all', 'around', 'me'], ['Fire', 'all', 'around', 'me'], ['Fire', 'surrounds', 'me'], ['Fire', 'in', 'building'], ['i', 'am', 'caught', 'in', 'fire'], ['Burning', 'my', 'house'], ['high', 'flames', 'burning', 'my', 'car'], ['Smoke', 'is', 'choking', 'me'], ['It', "'s", 'urgent', 'I', 'need', 'to', 'call', 'the', '[', 'fire', 'department', ']', ',', 'please'], ['We', 'need', 'the', '[', 'fire', 'brigade', ']', ',', 'please'], ['I', 'have', 'to', 'call', 'the', '[', 'firefighters', ']', ',', 'please'], ['A', 'fire', 'has', 'spread', 'in', 'the', 'room', ',', 'I', 'have', 'to', 'call', 'the', '[', 'firefighters', ']', ',', 'it', "'s", 'urgent', '!', '!', '!'], ['The', 'fire', 'alarm', 'has', 'been', 'triggered', ',', 'we', 'need', 'to', 'contact', 'the', '[', 'fire', 'department', ']', ',', 'we', 'need', 'an', 'help', '!', '!', '!'], ['I', 'have', 'to', 'call', 'the', '[', 'fire', 'chief', ']', ',', 'please'], ['I',

In [9]:
print(docs_y)

['Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood']


In [10]:
wrds = [stemmer.stem(w) for w in docs_x[13]]
print(wrds)

['a', 'fir', 'has', 'spread', 'in', 'the', 'room', ',', 'i', 'hav', 'to', 'cal', 'the', '[', 'firefight', ']', ',', 'it', "'s", 'urg', '!', '!', '!']


In [11]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))] # intialise the a list out_empty with 0's with len = length of labels

for x, doc in enumerate(docs_x): # creates tuples of docs_x's index and values such that the index of the docs_x elements are stores in x and the element values are stored in the variable doc
    
	bag = []
    
	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
            
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = np.array(training)
output = np.array(output)

In [12]:
print(training)
print(output)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [13]:
# neural network model
tf.compat.v1.reset_default_graph() # creates a fresh new empty graph by clearing the nodes and operations of any previous computation

net = tflearn.input_data(shape=[None, len(training[0])]) # None specifies that the input layer can accept a variable number of samples., no of neurons in input layer= 32
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") #  no of neurons = 3
net = tflearn.regression(net)
# net = tflearn.softmax_categorical_crossentropy(net,output)
model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
# neural network training
try:
	model.load("model.tflearn")
except:
    model = tflearn.DNN(net)
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")  

Training Step: 8999  | time: 0.031s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 64/67
Training Step: 9000  | time: 0.033s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 67/67
--
INFO:tensorflow:c:\programming languages\CYBERCUP\CyberCup\manya\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:c:\programming languages\CYBERCUP\CyberCup\manya\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:c:\programming languages\CYBERCUP\CyberCup\manya\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:c:\programming languages\CYBERCUP\CyberCup\manya\model.tflearn.meta
INFO:tensorflow:200


In [15]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]
	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return np.array(bag)

In [31]:
#chat interface
import speech_recognition as sr
import pyttsx3
import datetime

engine = pyttsx3.init('sapi5') #sapi5 is microsoft speech api
voices = engine.getProperty('voices') #main program
engine.setProperty('voice',voices[0].id)

def speak(audio):
    engine.say(audio)
    engine.runAndWait()

def chat():
    print("Start Talking with the Bot (say quit to stop!)")
    while True:
        r=sr.Recognizer()
        inp=[]
        with sr.Microphone() as source:
            print("Listening...")
            r.pause_threshold = 0.5
            audio = r.listen(source)
        try:
            print("Recognizing...")
            inp = r.recognize_google(audio, language='en-in')
            print(f"You: {inp}\n")
        
        except Exception as e:
            print(e)
            print("Say that again please...")
        
        if inp=="quit":
            return "None"
            
        
        results = model.predict([bag_of_words(inp, words)])[0]
        results_index=np.argmax(results)
        tag=labels[results_index]
        
        if results[results_index] > 0.5:
            for tg in dataset["data"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
                    speak(random.choice(responses))
                    print(random.choice(responses))
                    print("\n")
    
		#inp = input("You: ")
		#if inp.lower() == "quit":
		#	break

In [32]:
#start chatting
chat()

Start Talking with the Bot (say quit to stop!)
Listening...
Recognizing...
You: fire

Please consider the following measures- 
 1. Stay calm and do not panic 
 2. Sound the fire alarm and shout to alert others in the house 
 3. Call the local fire department immediately by dialing 101 or the emergency number in your area. Provide them with your address and details of the fire. 
 4. Close all doors and windows to contain the fire and prevent it from spreading. 
 5. Evacuate the house immediately and gather at a safe location outside.



Listening...
Recognizing...
You: super lunch

Please consider the following measures- 
 1. Stay calm and do not panic 
 2. Sound the fire alarm and shout to alert others in the house 
 3. Call the local fire department immediately by dialing 101 or the emergency number in your area. Provide them with your address and details of the fire. 
 4. Close all doors and windows to contain the fire and prevent it from spreading. 
 5. Evacuate the house immediatel

'None'